In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy import inspect, text
import pandas as pd

db_type = 'postgresql'
db_api = 'psycopg2'
postgres_host = os.getenv('POSTGRES_HOST')
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')
postgres_dbname = os.getenv('POSTGRES_DB')
postgres_port = 5432

print(postgres_host)
print(postgres_dbname)

In [ ]:
engine = create_engine(
    f'{db_type}+{db_api}://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_dbname}'
)

In [ ]:
engine.execution_options(isolation_level='AUTOCOMMIT').connect()

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
samuel_df = pd.read_csv('samuel_movies.csv')
tim_df = pd.read_csv('tim_movies.csv')

In [ ]:
print(samuel_df.dtypes)
print(tim_df.dtypes)

In [ ]:
samuel_df.to_sql('samuel_movies', engine, if_exists='replace')
tim_df.to_sql('tim_movies', engine, if_exists='replace')

In [ ]:
# Movies where both Samuel and Tim had a role.
sql_query_both = text('''SELECT sam."Title" from samuel_movies as sam, tim_movies as tim
WHERE sam."Title" = tim."Title"''')

with engine.connect() as conn:    
    data = conn.execute(sql_query_both)
    for d in data:
        print(d)

In [ ]:
sql_query_samuel_only = text('''
SELECT DISTINCT sam."Title", sam."Year" INTO Samuel_only from samuel_movies as sam, tim_movies as tim
WHERE sam."Title" != tim."Title"
''')

In [ ]:
sql_query_tim_only = text('''
SELECT DISTINCT tim."Title", tim."Year" INTO Tim_only FROM samuel_movies as sam, tim_movies as tim
WHERE sam."Title" != tim."Title"
''')

with engine.connect() as conn:    
    data = conn.execute(sql_query_tim_only)